--- Day 9: Rope Bridge ---

This rope bridge creaks as you walk along it. You aren't sure how old it is, or whether it can even support your weight.

It seems to support the Elves just fine, though. The bridge spans a gorge which was carved out by the massive river far below you.

You step carefully; as you do, the ropes stretch and twist. You decide to distract yourself by modeling rope physics; maybe you can even figure out where not to step.

Consider a rope with a knot at each end; these knots mark the head and the tail of the rope. If the head moves far enough away from the tail, the tail is pulled toward the head.

Due to nebulous reasoning involving Planck lengths, you should be able to model the positions of the knots on a two-dimensional grid. Then, by following a hypothetical series of motions (your puzzle input) for the head, you can determine how the tail will move.

Due to the aforementioned Planck lengths, the rope must be quite short; in fact, the head (H) and tail (T) must always be touching (diagonally adjacent and even overlapping both count as touching):

	....
	.TH.
	....

	....
	.H..
	..T.
	....

	...
	.H. (H covers T)
	...

If the head is ever two steps directly up, down, left, or right from the tail, the tail must also move one step in that direction so it remains close enough:

	.....    .....    .....
	.TH.. -> .T.H. -> ..TH.
	.....    .....    .....

	...    ...    ...
	.T.    .T.    ...
	.H. -> ... -> .T.
	...    .H.    .H.
	...    ...    ...

Otherwise, if the head and tail aren't touching and aren't in the same row or column, the tail always moves one step diagonally to keep up:

	.....    .....    .....
	.....    ..H..    ..H..
	..H.. -> ..... -> ..T..
	.T...    .T...    .....
	.....    .....    .....

	.....    .....    .....
	.....    .....    .....
	..H.. -> ...H. -> ..TH.
	.T...    .T...    .....
	.....    .....    .....

You just need to work out where the tail goes as the head follows a series of motions. Assume the head and the tail both start at the same position, overlapping.

For example:

R 4
U 4
L 3
D 1
R 4
D 1
L 5
R 2

This series of motions moves the head right four steps, then up four steps, then left three steps, then down one step, and so on. After each step, you'll need to update the position of the tail if the step means the head is no longer adjacent to the tail. Visually, these motions occur as follows (s marks the starting position as a reference point):

== Initial State ==

	......
	......
	......
	......
	H.....  (H covers T, s)

== R 4 ==

	......
	......
	......
	......
	TH....  (T covers s)

	......
	......
	......
	......
	sTH...

	......
	......
	......
	......
	s.TH..

	......
	......
	......
	......
	s..TH.

== U 4 ==

	......
	......
	......
	....H.
	s..T..

	......
	......
	....H.
	....T.
	s.....

	......
	....H.
	....T.
	......
	s.....

	....H.
	....T.
	......
	......
	s.....

== L 3 ==

	...H..
	....T.
	......
	......
	s.....

	..HT..
	......
	......
	......
	s.....

	.HT...
	......
	......
	......
	s.....

== D 1 ==

	..T...
	.H....
	......
	......
	s.....

== R 4 ==

	..T...
	..H...
	......
	......
	s.....

	..T...
	...H..
	......
	......
	s.....

	......
	...TH.
	......
	......
	s.....

	......
	....TH
	......
	......
	s.....

== D 1 ==

	......
	....T.
	.....H
	......
	s.....

== L 5 ==

	......
	....T.
	....H.
	......
	s.....

	......
	....T.
	...H..
	......
	s.....

	......
	......
	..HT..
	......
	s.....

	......
	......
	.HT...
	......
	s.....

	......
	......
	HT....
	......
	s.....

== R 2 ==

	......
	......
	.H....  (H covers T)
	......
	s.....

	......
	......
	.TH...
	......
	s.....

After simulating the rope, you can count up all of the positions the tail visited at least once. In this diagram, s again marks the starting position (which the tail also visited) and # marks other positions the tail visited:

	..##..
	...##.
	.####.
	....#.
	s###..

So, there are 13 positions the tail visited at least once.

Simulate your complete hypothetical series of motions. How many positions does the tail of the rope visit at least once?


In [17]:
import numpy as np

In [1]:
with open('input.txt') as f:
	movs = f.read().splitlines()

In [2]:
with open('test.txt') as f:
	test = f.read().splitlines()

In [4]:
def parse(movs):
	# Parse the moves (input) into a list of tuples
	# (direction, length), [(char, int)]
	# direction is: 'U', 'D', 'L', or 'R' (up, down, left, right)
	# length is an integer
	parsed = []
	for mov in movs:
		dir = mov[0]
		length = int(mov[1:])
		parsed.append((dir, length))
	return parsed

In [19]:
def touching(pos1, pos2):
	# Return True if the two positions are touching
	# Diagonal touching is not allowed
	x1, y1 = pos1
	x2, y2 = pos2

	if abs(x1 - x2)==1 and abs(y1 - y2) == 1:
		return True
	elif abs(x1 - x2) + abs(y1 - y2) <= 1:
		return True
	else:
		return False

In [36]:
def part1(movs, pos[0]=(0,0), T_pos=(0,0)):
	passed = {(0, 0)}
	for mov in parse(movs):
		direction, length = mov
		for i in range(length):
			# Head (pos[0]) movement
			if direction == 'U':
				H_pos = (H_pos[0], H_pos[1] + 1)
			elif direction == 'D':
				H_pos = (H_pos[0], H_pos[1] - 1)
			elif direction == 'L':
				H_pos = (H_pos[0] - 1, H_pos[1])
			elif direction == 'R':
				H_pos = (H_pos[0] + 1, H_pos[1])

			# Check if they are touching (diagonal touching is allowed)
			if not touching(H_pos, T_pos):
				x_dis = H_pos[0] - T_pos[0] 
				y_dis = H_pos[1] - T_pos[1]

				# First: Diagonal moves
				if abs(x_dis)>=1 and abs(y_dis)>= 1:
					T_pos = (T_pos[0] + np.sign(x_dis), T_pos[1] + np.sign(y_dis))
				# Second: Horizontal moves
				elif abs(x_dis) == 2:
					T_pos = (T_pos[0] + np.sign(x_dis), T_pos[1])
				# Third: Vertical moves
				elif abs(y_dis) == 2:
					T_pos = (T_pos[0], T_pos[1] + np.sign(y_dis))
				else:
					print('Error')

			passed.add(T_pos)
	return len(passed)


In [37]:
part1(movs)

6470

# --- Part 2 ---

Same as before but with a 10 knots rope. Each knot follows the previous one. Same rules as before.

In [40]:
with open('test2.txt') as f:
	test2 = f.read().splitlines()

In [43]:
def part2(movs, knots = 10):
	pos = [(0,0) for i in range(knots)]
	passed = {(0, 0)}
	for mov in parse(movs):
		direction, length = mov
		for i in range(length):
			# Head (H_pos) movement
			H_pos = pos[0]
			if direction == 'U':
				H_pos = (H_pos[0], H_pos[1] + 1)
			elif direction == 'D':
				H_pos = (H_pos[0], H_pos[1] - 1)
			elif direction == 'L':
				H_pos = (H_pos[0] - 1, H_pos[1])
			elif direction == 'R':
				H_pos = (H_pos[0] + 1, H_pos[1])
			pos[0] = H_pos

			for i in range(1, knots):
				H_pos = pos[i-1]
				T_pos = pos[i]

				# Check if they are touching (diagonal touching is allowed)
				if not touching(H_pos, T_pos):
					x_dis = H_pos[0] - T_pos[0] 
					y_dis = H_pos[1] - T_pos[1]

					# First: Diagonal moves
					if abs(x_dis)>=1 and abs(y_dis)>= 1:
						T_pos = (T_pos[0] + np.sign(x_dis), T_pos[1] + np.sign(y_dis))
					# Second: Horizontal moves
					elif abs(x_dis) == 2:
						T_pos = (T_pos[0] + np.sign(x_dis), T_pos[1])
					# Third: Vertical moves
					elif abs(y_dis) == 2:
						T_pos = (T_pos[0], T_pos[1] + np.sign(y_dis))
					else:
						print('Error')
				
				pos[i] = T_pos
						
			passed.add(T_pos)
			
	return len(passed)


In [45]:
part2(movs, 10)

2658